In [9]:
import os
import numpy as np
import librosa
from hmmlearn import hmm
from pykalman import KalmanFilter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [10]:
a=int(2251262633%15+1)
print(a)

9


In [11]:
DATA_DIR = r"C:\Users\admin\OneDrive\Time_series\data"  # Đảm bảo thư mục này chứa 'off' và 'up'
ALLOWED_LABELS = {'off', 'up'}  # Chỉ nhận dạng 2 từ này
N_MFCC =16
N_HMM_STATES = 3
def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC)
    return mfcc.T

In [12]:
def apply_kalman(mfcc):
    kf = KalmanFilter(initial_state_mean=mfcc[0], n_dim_obs=mfcc.shape[1])
    smoothed_mfcc, _ = kf.smooth(mfcc)
    return smoothed_mfcc

In [13]:
def train_hmm_model(feature_list):
    all_features = np.vstack(feature_list)
    model = hmm.GaussianHMM(n_components=N_HMM_STATES, covariance_type='diag', n_iter=100)
    model.fit(all_features)
    return model

In [14]:
def load_dataset(apply_kalman_filter=False):
    X = []
    y = []
    for label in os.listdir(DATA_DIR):
        if label not in ALLOWED_LABELS:
            continue  # Bỏ qua các thư mục không phải 'off' hoặc 'up'
        label_path = os.path.join(DATA_DIR, label)
        if not os.path.isdir(label_path):
            continue
        for filename in os.listdir(label_path):
            if filename.lower().endswith(".wav"):
                file_path = os.path.join(label_path, filename)
                mfcc = extract_mfcc(file_path)
                if apply_kalman_filter:
                    mfcc = apply_kalman(mfcc)
                X.append(mfcc)
                y.append(label)
    return X, y

In [15]:
def predict(models, sample):
    scores = {label: model.score(sample) for label, model in models.items()}
    return max(scores, key=scores.get)

# === ĐÁNH GIÁ ===
def evaluate_model(X_test, y_test, models):
    predictions = [predict(models, sample) for sample in X_test]
    return accuracy_score(y_test, predictions), predictions


In [16]:
def main():
    print("==== NHẬN DẠNG VỚI HMM ====")
    X, y = load_dataset(apply_kalman_filter=False)
    print(f"Tổng mẫu âm thanh (không dùng Kalman): {len(X)}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    hmm_models = {}
    for label in set(y_train):
        features = [x for x, label_y in zip(X_train, y_train) if label_y == label]
        hmm_models[label] = train_hmm_model(features)

    acc_hmm, pred_hmm = evaluate_model(X_test, y_test, hmm_models)
    print(f" Độ chính xác HMM: {acc_hmm:.2%}")

    print("\n==== NHẬN DẠNG VỚI HMM + KALMAN ====")
    X_kf, y_kf = load_dataset(apply_kalman_filter=True)
    print(f"Tổng mẫu âm thanh (dùng Kalman): {len(X_kf)}")
    X_train_kf, X_test_kf, y_train_kf, y_test_kf = train_test_split(X_kf, y_kf, test_size=0.3, random_state=42)

    hmm_kalman_models = {}
    for label in set(y_train_kf):
        features = [x for x, label_y in zip(X_train_kf, y_train_kf) if label_y == label]
        hmm_kalman_models[label] = train_hmm_model(features)

    acc_kf, pred_kf = evaluate_model(X_test_kf, y_test_kf, hmm_kalman_models)
    print(f" Độ chính xác HMM + Kalman: {acc_kf:.2%}")

    print("\n==== KẾT THÚC ====")

if __name__ == "__main__":
    main()

==== NHẬN DẠNG VỚI HMM ====
Tổng mẫu âm thanh (không dùng Kalman): 4737
 Độ chính xác HMM: 66.95%

==== NHẬN DẠNG VỚI HMM + KALMAN ====
Tổng mẫu âm thanh (dùng Kalman): 4737
 Độ chính xác HMM + Kalman: 67.30%

==== KẾT THÚC ====
